In [16]:
import tensorflow as tf
import pyunpack
import wget
import pandas as pd
import numpy as np
from pandas import DataFrame, Series, Timestamp
from numpy import ndarray

In [2]:
import sys
sys.path.insert(0, '..')

In [3]:
df = pd.read_csv(r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\electricity\data\electricity\LD2011_2014.txt', index_col=0, sep=';', decimal=',')

In [4]:
df.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_361,MT_362,MT_363,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369,MT_370
2011-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 01:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df.shape

(140256, 370)

In [7]:
#preprocess
df.index: Series = pd.to_datetime(df.index)
df.sort_index(inplace=True)
# Used to determine the start and end dates of a series
output: DataFrame = df.resample('1h').mean().replace(0., np.nan)

In [8]:
output.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_361,MT_362,MT_363,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369,MT_370
2011-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
earliest_time: Timestamp = output.index.min()
earliest_time

Timestamp('2011-01-01 00:00:00', freq='H')

In [17]:
df_list = []
for label in output:
    print('Processing {}'.format(label))
    srs: Series = output[label]
    start_date: Timestamp = min(srs.fillna(method='ffill').dropna().index)
    end_date: Timestamp = max(srs.fillna(method='bfill').dropna().index)
    active_range: ndarray = (srs.index >= start_date) & (srs.index <= end_date)
    srs: Series = srs[active_range].fillna(0.)
    
    tmp: DataFrame = pd.DataFrame({'power_usage': srs})
    date: Series = tmp.index
    tmp['t']: Series = (date - earliest_time).seconds / 60 / 60 + (
                date - earliest_time).days * 24
    tmp['days_from_start'] = (date - earliest_time).days
    tmp['categorical_id'] = label
    tmp['date'] = date
    tmp['id'] = label
    tmp['hour'] = date.hour
    tmp['day'] = date.day
    tmp['day_of_week'] = date.dayofweek
    tmp['month'] = date.month
    
    df_list.append(tmp)

output: DataFrame = pd.concat(df_list, axis=0, join='outer').reset_index(drop=True)

output['categorical_id']: Series = output['id'].copy()
output['hours_from_start']: Series = output['t']
output['categorical_day_of_week']: Series = output['day_of_week'].copy()
output['categorical_hour']: Series = output['hour'].copy()
# Filter to match range used by other academic papers
output: DataFrame = output[(output['days_from_start'] >= 1096)
                & (output['days_from_start'] < 1346)].copy()

Processing MT_001
Processing MT_002
Processing MT_003
Processing MT_004
Processing MT_005
Processing MT_006
Processing MT_007
Processing MT_008
Processing MT_009
Processing MT_010
Processing MT_011
Processing MT_012
Processing MT_013
Processing MT_014
Processing MT_015
Processing MT_016
Processing MT_017
Processing MT_018
Processing MT_019
Processing MT_020
Processing MT_021
Processing MT_022
Processing MT_023
Processing MT_024
Processing MT_025
Processing MT_026
Processing MT_027
Processing MT_028
Processing MT_029
Processing MT_030
Processing MT_031
Processing MT_032
Processing MT_033
Processing MT_034
Processing MT_035
Processing MT_036
Processing MT_037
Processing MT_038
Processing MT_039
Processing MT_040
Processing MT_041
Processing MT_042
Processing MT_043
Processing MT_044
Processing MT_045
Processing MT_046
Processing MT_047
Processing MT_048
Processing MT_049
Processing MT_050
Processing MT_051
Processing MT_052
Processing MT_053
Processing MT_054
Processing MT_055
Processing

In [18]:
output.head(20)

,power_usage,t,days_from_start,categorical_id,date,id,hour,day,day_of_week,month,hours_from_start,categorical_day_of_week,categorical_hour
17544,2.538071,26304.0,1096,MT_001,2014-01-01 00:00:00,MT_001,0,1,2,1,26304.0,2,0
17545,2.855330,26305.0,1096,MT_001,2014-01-01 01:00:00,MT_001,1,1,2,1,26305.0,2,1
17546,2.855330,26306.0,1096,MT_001,2014-01-01 02:00:00,MT_001,2,1,2,1,26306.0,2,2
17547,2.855330,26307.0,1096,MT_001,2014-01-01 03:00:00,MT_001,3,1,2,1,26307.0,2,3
17548,2.538071,26308.0,1096,MT_001,2014-01-01 04:00:00,MT_001,4,1,2,1,26308.0,2,4
17549,2.538071,26309.0,1096,MT_001,2014-01-01 05:00:00,MT_001,5,1,2,1,26309.0,2,5
17550,3.172589,26310.0,1096,MT_001,2014-01-01 06:00:00,MT_001,6,1,2,1,26310.0,2,6
17551,2.855330,26311.0,1096,MT_001,2014-01-01 07:00:00,MT_001,7,1,2,1,26311.0,2,7
17552,1.269036,26312.0,1096,MT_001,2014-01-01 08:00:00,MT_001,8,1,2,1,26312.0,2,8
17553,0.317259,26313.0,1096,MT_001,2014-01-01 09:00:00,MT_001,9,1,2,1,26313.0,2,9


In [19]:
output.tail()

,power_usage,t,days_from_start,categorical_id,date,id,hour,day,day_of_week,month,hours_from_start,categorical_day_of_week,categorical_hour
10461482,20824.324324,32299.0,1345,MT_370,2014-09-07 19:00:00,MT_370,19,7,6,9,32299.0,6,19
10461483,19527.027027,32300.0,1345,MT_370,2014-09-07 20:00:00,MT_370,20,7,6,9,32300.0,6,20
10461484,20202.702703,32301.0,1345,MT_370,2014-09-07 21:00:00,MT_370,21,7,6,9,32301.0,6,21
10461485,19851.351351,32302.0,1345,MT_370,2014-09-07 22:00:00,MT_370,22,7,6,9,32302.0,6,22
10461486,20135.135135,32303.0,1345,MT_370,2014-09-07 23:00:00,MT_370,23,7,6,9,32303.0,6,23
